# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [411]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

conn = sqlite3.Connection("data2.sqlite")

In [412]:
# Your code here; create the connection

SELECT p.productName, COUNT(DISTINCT o.customerNumber) AS total_num_of_people
FROM products p 
JOIN orders o on o.orderNumber = od.orderNumber
JOIN orderdetails od on od.productCode = p.productCode
GROUP BY
p.productName
ORDER BY 
total_num_of_people DES

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3677879927.py, line 3)

### Q.1 Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [ ]:
# Your code here
q0 = """
SELECT 
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
)
;
"""
pd.read_sql(q0, conn)

### Q.2 Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [413]:
# Your code here
q1 = """

SELECT p.productName, p.productCode, COUNT(od.orderNumber) AS total_num_of_orders
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY p.productName, p.productCode
ORDER BY total_num_of_orders DESC;


"""
pd.read_sql(q1, conn)

,productName,productCode,total_num_of_orders
0,1992 Ferrari 360 Spider red,S18_3232,53
1,18th Century Vintage Horse Carriage,S18_3136,28
2,1900s Vintage Bi-Plane,S24_2841,28
3,1900s Vintage Tri-Plane,S24_4278,28
4,1913 Ford Model T Speedster,S18_2949,28
...,...,...,...
104,1970 Chevy Chevelle SS 454,S24_1046,25
105,1999 Indy 500 Monte Carlo SS,S18_2870,25
106,2002 Chevy Corvette,S24_3432,25
107,1952 Citroen-15CV,S24_2887,24


### Q.3 Select the Product Name and the  Total Number of People Who Have Ordered Each Product
Sort the results in descending order.

**A quick note on the SQL  `SELECT DISTINCT` statement:**

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [414]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

q2 = """

SELECT p.productName, COUNT(DISTINCT o.customerNumber) AS total_num_of_people
FROM products p 
JOIN orders o on o.orderNumber = od.orderNumber
JOIN orderdetails od on od.productCode = p.productCode
GROUP BY
p.productName
ORDER BY 
total_num_of_people DESC

"""
pd.read_sql(q2, conn)

,productName,total_num_of_people
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


### Q.4 Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [415]:
# Your code here
q3 = """

SELECT e.employeeNumber, e.firstName, e.lastName, of.city, e.officeCode, p.productName, p.productCode, COUNT (DISTINCT c.customerNumber)
FROM employees e
JOIN offices of ON of.officeCode = e.officeCode
JOIN customers c ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN orders o ON o.customerNumber = c.customerNumber
JOIN orderdetails od ON od.orderNumber = o.orderNumber
JOIN products p ON p.productCode = od.productCode
GROUP BY
e.employeeNumber
HAVING COUNT(DISTINCT c.customerNumber) < 20
"""

pd.read_sql(q3, conn)

,employeeNumber,firstName,lastName,city,officeCode,productName,productCode,COUNT (DISTINCT c.customerNumber)
0,1165,Leslie,Jennings,San Francisco,1,1958 Setra Bus,S12_1666,6
1,1166,Leslie,Thompson,San Francisco,1,1917 Grand Touring Sedan,S18_1749,6
2,1188,Julie,Firrelli,Boston,2,1952 Alpine Renault 1300,S10_1949,6
3,1216,Steve,Patterson,Boston,2,2001 Ferrari Enzo,S12_1108,6
4,1286,Foon Yue,Tseng,NYC,3,2001 Ferrari Enzo,S12_1108,6
5,1323,George,Vanauf,NYC,3,1969 Harley Davidson Ultimate Chopper,S10_1678,8
6,1337,Loui,Bondur,Paris,4,1952 Alpine Renault 1300,S10_1949,6
7,1370,Gerard,Hernandez,Paris,4,1965 Aston Martin DB5,S18_1589,7
8,1401,Pamela,Castillo,Paris,4,1972 Alfa Romeo GTA,S10_4757,10
9,1501,Larry,Bott,London,7,1972 Alfa Romeo GTA,S10_4757,8


### Q.5 Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [416]:
# Your code here
q4 = """
SELECT e.employeeNumber, e.lastName, e.firstName, AVG(c.creditLimit), COUNT(c.customerNumber) AS num_of_customers
FROM employees e
JOIN customers c ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY e.employeeNumber
HAVING AVG(creditLimit) > 15000
"""

pd.read_sql(q4, conn)

,employeeNumber,lastName,firstName,AVG(c.creditLimit),num_of_customers
0,1165,Jennings,Leslie,100433.333333,6
1,1166,Thompson,Leslie,65266.666667,6
2,1188,Firrelli,Julie,73916.666667,6
3,1216,Patterson,Steve,81533.333333,6
4,1286,Tseng,Foon Yue,66614.285714,7
5,1323,Vanauf,George,80887.500000,8
6,1337,Bondur,Loui,86233.333333,6
7,1370,Hernandez,Gerard,91785.714286,7
8,1401,Castillo,Pamela,81340.000000,10
9,1501,Bott,Larry,91187.500000,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!